In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [ ]:
trian_csv=pd.read_csv('../input/histopathologic-cancer-detection/train_labels.csv')

In [ ]:
trian_csv.head()

In [ ]:
trian_csv.shape

In [ ]:
trian=trian_csv[trian_csv['label']==0]

In [ ]:
trian.shape

In [ ]:
trian_1=trian.iloc[:80000,:]

In [ ]:
trian_1.shape

In [ ]:
trian_2=trian.iloc[80000:87000,:]

In [ ]:
trian_2.shape

In [ ]:
fog=trian_csv[trian_csv['label']==1]

In [ ]:
fog.head()

In [ ]:
fog.shape

In [ ]:
fog_1=fog.iloc[:80000,:]

In [ ]:
fog_2=fog.iloc[80000:87000,:]

In [ ]:
fog_2.shape

In [ ]:
trian_csv=pd.concat([trian_1,fog_1])

In [ ]:
trian_csv.shape

In [ ]:
test_csv=pd.concat([trian_2,fog_2])

In [ ]:
test_csv.shape

In [ ]:
trian_csv.isnull().sum()

In [ ]:
sns.countplot(x='label',data=trian_csv)

In [ ]:
trian_csv["id"] = trian_csv["id"].apply(lambda x: x + ".tif")
trian_csv.head()

In [ ]:
trian_csv['label'] = trian_csv['label'].astype('str')

In [ ]:
test_csv.isnull().sum()

In [ ]:
sns.countplot(x='label',data=test_csv)

In [ ]:
test_csv['label'] = test_csv['label'].astype('str')

In [ ]:
test_csv["id"] = test_csv["id"].apply(lambda x: x + ".tif")
test_csv.head()

In [ ]:
test_csv.shape

In [ ]:
from tensorflow.keras.applications.inception_v3 import  preprocess_input ,InceptionV3

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input , MobileNet

In [ ]:
import tensorflow as tf

In [ ]:


datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.0,
           # preprocessing_function=preprocess_input,
            rotation_range=10,
            horizontal_flip=True,
            vertical_flip=True,
            validation_split = 0.1
            )

In [ ]:
train_generator = datagen.flow_from_dataframe(
            dataframe=trian_csv,
            directory="../input/histopathologic-cancer-detection/train/",
            x_col="id",
            y_col="label",
            subset="training",
            batch_size=128,
            seed=42,
            shuffle=True,
            class_mode="binary",
            target_size=(64,64))

In [ ]:
image=cv2.imread('../input/histopathologic-cancer-detection/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')
image.shape

In [ ]:
classes=list(train_generator.class_indices.keys())
plt.figure(figsize=(20,20))
for X_batch, y_batch in train_generator:
    # create a grid of 3x3 images
    for i in range(0,36):
        plt.subplot(6,6,i+1)
        plt.imshow(X_batch[i])
    # show the plot
    plt.show()
    break

In [ ]:
datagen_valid = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.,
   # preprocessing_function=preprocess_input,
            validation_split = 0.1
            )

In [ ]:
valid_generator = datagen_valid.flow_from_dataframe(
            dataframe=trian_csv,
            directory="../input/histopathologic-cancer-detection/train",
            x_col="id",
            y_col="label",
            subset="validation",
            batch_size=128,
            seed=42,
            shuffle=False,
            class_mode="binary",
            target_size=(64,64))

In [ ]:
plt.figure(figsize=(20,20))
for X_batch, y_batch in valid_generator:
    # create a grid of 3x3 images
    for i in range(0,36):
        plt.subplot(6,6,i+1)
        plt.imshow(X_batch[i])
    # show the plot
    plt.show()
    break

In [ ]:
datagen_test = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.
   # preprocessing_function=preprocess_input
            )

In [ ]:
test_generator = datagen_test.flow_from_dataframe(
            dataframe=test_csv,
            directory="../input/histopathologic-cancer-detection/train",
            x_col="id",
            y_col="label",
            batch_size=128,
            seed=42,
            shuffle=False,
            class_mode="binary",
            target_size=(64,64))

In [ ]:
plt.figure(figsize=(20,20))
for X_batch, y_batch in test_generator:
    # create a grid of 3x3 images
    for i in range(0,36):
        plt.subplot(6,6,i+1)
        plt.imshow(X_batch[i])
        plt.grid(visible=None)
    # show the plot
    plt.show()
    break

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense ,Flatten ,Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization ,GlobalMaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau ,ModelCheckpoint

In [ ]:
#optimizer
optimizer_adam=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.99)

In [ ]:
#callbacks

EarlyStop=EarlyStopping(patience=10,restore_best_weights=True)
Reduce_LR=ReduceLROnPlateau(monitor='val_accuracy',verbose=2,factor=0.5,min_lr=0.00001)
model_check=ModelCheckpoint('model.hdf5',monitor='val_loss',verbose=1,save_best_only=True)
callback=[EarlyStop , Reduce_LR,model_check]

In [ ]:
model=Sequential([
                  Conv2D(64,3,activation='relu',kernel_initializer='he_normal',input_shape=(64,64,3)),
                  BatchNormalization(),
                  Conv2D(128,3,activation='relu',kernel_initializer='he_normal'),
                  BatchNormalization(),
                  MaxPooling2D(3),
                  Conv2D(256,3,activation='relu'),
                  BatchNormalization(),
                  MaxPooling2D(3),
                  Conv2D(512,3,padding='valid',activation='relu'),
                  BatchNormalization(),
                  GlobalMaxPool2D(),
                  Flatten(),
                  Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                  BatchNormalization(),
                  Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                  BatchNormalization(),
                  Dense(1,activation='sigmoid',kernel_initializer='glorot_normal')
                  
])

model.compile(optimizer=optimizer_adam,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:

Mb=MobileNet(include_top=False,weights='imagenet',input_shape=(64,64,3))
Mb.trainable=False
model=Sequential([
                  Mb,
                  Flatten(),
                  Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                  BatchNormalization(),
                  Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                  BatchNormalization(),
                  Dense(1,activation='sigmoid',kernel_initializer='glorot_normal')
                  
])

model.compile(optimizer=optimizer_adam,loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
history=model.fit(train_generator,validation_data=valid_generator,epochs=100,
                  steps_per_epoch=len(train_generator)//64,validation_steps=len(valid_generator)//64,
                  callbacks=callback, verbose=1)


In [ ]:
model.evaluate(test_generator)

In [ ]:
#plotting training values
import seaborn as sns
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='green', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
predictions = model.predict(test_generator)

In [ ]:
predictions[0]

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_generator.classes, predictions.round())

sns.heatmap(cm,  annot=True, fmt="d" ,cmap="YlGnBu")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_generator.classes, predictions.round()))